## Arcpy: Working with Cursors and Geometries  

In this exercise we'll walk through the Search Cursor and Update Cursor, and complete some light attribute and geometry editing. 

  
Script by: Stefan Leyk  
Updated 1/2022 by: Phil White

In [ ]:
import arcpy
import traceback
import sys
from arcpy import env

Set your workspace to our Week2 Data directory:

In [ ]:
env.workspace = r'C:\your\path\GEOG_4303\Week2\data'

Just as a reminder to what files we have in this drive:

In [ ]:
print arcpy.ListFeatureClasses()

Now set your workspace back to the Week2 directory:

In [ ]:
env.workspace = r'C:\your\path\GEOG_4303\Week2'

Okay, let's create variables for our input shapefile and a new output shapefile:

In [ ]:
inFC = r'/data/lyons_mrd.shp'

In [ ]:
copyFC = r'/results/copy_lyons_mrd.shp'

Now, we can use this if/else statement to check if our output file exists, and delete it if it does. This will be useful later on if we need to rerun sections of code.

In [ ]:
if arcpy.Exists(copyFC):
    arcpy.management.Delete(copyFC)
    print 'existing output file DESTROYED.'
else:
    print 'nothing to blow up, sorry.'

This snippet uses `management.Copy()` to createa fresh copy of our input Lyons roads shapefile:

In [ ]:
arcpy.management.Copy(inFC,copyFC)

Check the fields of this shapefile using `.ListFields()`:

In [ ]:
fields = arcpy.ListFields(copyFC)
for field in fields:
    print field.name

### Search Cursor

Let's take a sec to read about the data access search cursor: [https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/searchcursor-class.htm](https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/searchcursor-class.htm)  

SearchCursor establishes **read-only** access to the records returned from a feature class or table.  

*Basically, search cursor allows you to loop over values of an attribute field.*

In [ ]:
sCursor = arcpy.da.SearchCursor(copyFC,['FID','ZIPL'])

print sCursor.next()
print sCursor.next()
print sCursor.next()

del sCursor

What data type is returned? https://www.w3schools.com/python/python_datatypes.asp  

How would we return just the zip codes without the index?

In [ ]:
sCursor = arcpy.da.SearchCursor(copyFC,['FID','ZIPL'])
for row in sCursor:
    print row
    
del sCursor    

What if we wanted to count occurrences of the zip code 80540? 

In [ ]:
sCursor = arcpy.da.SearchCursor(copyFC,['FID','ZIPL'])

count = 0

for row in sCursor:
    if row[1] == '80540':
        count += 1
        print 'Zip at ' + str(row[0]) + 'is ' + str(row[1])
        
print 'Zip code 80540 occurs ' + str(count) + ' times.'

del sCursor

### Now let's look at Geometry.  


Again, see docs: https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/searchcursor-class.htm  

From here on, we'll use `with` to instantiate the sCursor:

In [ ]:
with arcpy.da.SearchCursor(copyFC,['SHAPE@']) as sCursor:

    row = sCursor.next()

    print row

Okay, so we know it's a polyline... a polyline is a geometry type...  Not much else to see here...

Take a look at the ArcPy Geometry class: https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-classes/geometry.htm 

In [ ]:
with arcpy.da.SearchCursor(copyFC,['SHAPE@']) as sCursor:

    row = sCursor.next()

    pLine = row[0] 

    print pLine.isMultipart 
    #print pLine.partCount
    #print pLine.pointCount
    #print pLine.length
    #print pLine.getPart()

Why are we getting these results? What does this mean?  

`.getPart()` returns an [array](https://www.w3schools.com/python/python_arrays.asp) that tells us information about the different parts... in this case the start and end points of our polyline. If you want one of these parts, use it's index position...  

As you run the next code block, run, examine results, uncomment the next line, re-run, until you've worked through the whole thing. What you're doing here is accessing the x,y coordinates for the first point in a polyline.  

In [ ]:
with arcpy.da.SearchCursor(copyFC,['SHAPE@']) as sCursor:

    row = sCursor.next()

    pLine = row[0] 

    pLineArray = pLine.getPart(0) #this returns the first index

    print pLineArray

    #point1 = pLineArray[1]

    #print point1

    #print point1.X
    #print point1.Y

That's all good... but let's do this in a smarter way... Make a `for` loop!  

In [ ]:
with arcpy.da.SearchCursor(copyFC,['SHAPE@']) as sCursor:
    row = sCursor.next()
    pLine = row[0] 
    pLineArray = pLine.getPart(0)

    for p in pLineArray:
        print str(p.X) + ', ' + str(p.Y) 

There are LOTS of geometry methods. `.touches()` for example returns topology information:  

**What should the following return?**  

Again, see Geometry methods: https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-classes/geometry.htm

In [ ]:
with arcpy.da.SearchCursor(copyFC,['SHAPE@']) as sCursor:
    row = sCursor.next()
    pLine = row[0] 
    pLineArray = pLine.getPart(0)
    point1 = pLineArray[1]
    
    #Does point1 touch the polyline?
    print point1.touches(pLine)

We can use the `.Polyline()` geometry method to create a new line from one of the line segments:

In [ ]:
with arcpy.da.SearchCursor(copyFC,['SHAPE@']) as sCursor:
    row = sCursor.next()
    pLine = row[0] #take the first polyline
    pLineArray = pLine.getPart(0) #take the first part of the first polyline
    
    newPolyLine = arcpy.Polyline(pLineArray) #create a new polyline from the existing segment

In [ ]:
newPolyLine

Hm... Now what?   

### Update Cursor  

So far we've been reading and examining features of our shapefile using SearchCursor...  We can accomplish things when reading-only, but if you want to update (make changes to the file) you want to use the UpdateCursor.  

Docs: https://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-data-access/updatecursor-class.htm  

UpdateCursor establishes **read-write** access to records returned from a feature class or table.  

First let's use the update cursor to populate a new attribute field:


In [ ]:
#first add a field:
arcpy.management.AddField(copyFC,'ZIP2', 'LONG')
#this new field is named ZIP2 and is a LONG type

In [ ]:
#Now we will populate the field:
with arcpy.da.UpdateCursor(copyFC, ['FID','ZIPL','ZIP2']) as uCursor: # this activates the cursor, working on the three attributes we specified
    for row in uCursor:
        if row[1] == '80540':
            row[2] = 3562
            print 'Zip2 at ' + str(row[0]) + ' is now ' + str(row[2])
        uCursor.updateRow(row)

Did it work? Check it with SearchCursor:

In [ ]:
with arcpy.da.SearchCursor(copyFC, ['FID','ZIPL','ZIP2']) as sCursor:
    for row in sCursor:
        print row

Neat...  

Let's do some more UpdateCursor, this time working on geometry. 

We will shift a line's end point 2000 meters to the east... but first, what reference system is this in? 

In [ ]:
arcpy.Describe(copyFC).spatialReference.name

Think for a sec how UTM works... https://www.usgs.gov/faqs/how-are-utm-coordinates-measured-usgs-topographic-maps#:~:text=The%20UTM%20  

**Now**, work through the update cursor block below. Run it, examine the results, then uncomment the next line, re-run, and examine what it does, continue through each line. You will 

In [ ]:
with arcpy.da.UpdateCursor(copyFC,['SHAPE@']) as uCursor:
    row = uCursor.next()
    pLine = row[0]
    print pLine.pointCount
    #pLineArr = pLine.getPart(0)
    #p1 = pLineArr[0]
    #print 'old point 1 = ', p1.X
    #p1.X = p1.X+2000
    #print 'new point 1 = ', p1.X
    
    #newPolyLine = arcpy.Polyline(pLineArr)
    #uCursor.updateRow([newPolyLine])

What happens if we run this again?

Now, let's do the same except move the x values in all of the lines in the copyFC shepfile:

In [ ]:
if arcpy.Exists(copyFC):
    arcpy.management.Delete(copyFC)
    print 'existing output file DESTROYED.'
else:
    print 'nothing to blow up, sorry.'

In [ ]:
arcpy.management.Copy(inFC,copyFC)

In [ ]:
with arcpy.da.UpdateCursor(copyFC,['SHAPE@']) as uCursor:
    row = uCursor.next()
    pLine = row[0]
    pLineArr = pLine.getPart(0)
    for i in range(len(pLineArr)):
        p=pLineArr[i]
        print 'old values of point' + str(i) +' : ', p
        p.X = p.X+2000
        print 'new values of point' + str(i) +' : ', p
        
    newPolyLine=arcpy.Polyline(pLineArr)
    uCursor.updateRow([newPolyLine])
del pLineArr 

Did it work? Open up the input and output files in ArcMap to view the results.